In [1]:
import requests
import base64
import urllib
from sha256 import SHA256
import os
import sha256sum

In [2]:
email = 'solujan@gmail.com'
url = f'https://ciberseguridad.diplomatura.unc.edu.ar/cripto/secret-prefix-mac/{email}/challenge'
# Solicitud
request = requests.get(url)
# Body de la respuesta
message = request.content
message

b'user=solujan@gmail.com&action=show&mac=4bb1e69e282cd62a9b8c87d199d30d0813dce6261b6f9d632b39749135f63381'

Vemos que en este ejemplo hay tres pares clave-valor:

* user=solujan@gmail.com
* action=show
* mac=4bb1e69e282cd62a9b8c87d199d30d0813dce6261b6f9d632b39749135f63381

**Objetivo : La falsificación debe ser una query string válida que incluya el par admin=true.**



* Para calcular el MAC, se construye un mensaje formado por la concatenación del resto de los pares clave-valor, ordenados alfabéticamente por clave, y eliminando los símbolos = y &. En el caso anterior, el mensaje sería:

```
actionshowuseruser@example.com
```

* Sobre este mensaje se calcula un MAC de la siguiente forma:

```
MAC = SHA-256(secreto || mensaje)
```

donde:

* mensaje es el mensaje conformado como se describe más arriba.
* secreto es una clave secreta de 16 bytes.
* || denota concatenación

In [3]:
original_mac = message[39:]
original_mac

b'4bb1e69e282cd62a9b8c87d199d30d0813dce6261b6f9d632b39749135f63381'

In [4]:
mensaje = message[:34].decode()

mensaje = mensaje.split("&")
mensaje_sorted = sorted(mensaje)
mensaje_clean = "".join(mensaje_sorted).replace("=", "")
mensaje_clean

'actionshowusersolujan@gmail.com'

In [5]:
mensaje_bytes = mensaje_clean.encode()
mensaje_bytes

b'actionshowusersolujan@gmail.com'

In [22]:
len(mensaje_bytes)

31

In [50]:
len(original_mac)

64

La longitud del mensaje original es de 31 bytes (31 * 8 = 248 bits)

Hay que completar el bloque de 64 bytes (512 bits)

Para ello es necesario agregar un 1 y luego completar con 0 hasta los

In [64]:
def pad(message):
  #suponemos que mensaje secreto tiene len dinamico:
  # | ? | ? | ? | ? | a | c | t | i | o | n | s | h | o |... | o | m | 80 | 0 | 0 | 0 | 0 | ... | len_total
  key_len_bytes = 16
  message_len_bytes = len(message)
  len_total_bits = (message_len_bytes + key_len_bytes)*8
  len_bytes = len_total_bits.to_bytes(2, byteorder = 'big')

  padded = message + b'\x80'.ljust(62- (len(message) + 16), b'\x00') + b'\x01' + b'\x78'
  print(f'padded: {padded}\nlen_bytes: {len_bytes.hex()}\nlen {len(padded)+16}\n{len_total_bits}')
  return padded

In [65]:
#original_hash = sha256sum.SHA256(mensaje_bytes).digest()
padded = pad(mensaje_bytes)
extension = b'admintrueusersolujan@gmail.com'
new_mac = sha256sum.SHA256(extension, original_mac, len(padded)+16).digest()
#expected = sha256sum.SHA256(padded + extension).digest()
#print(expected.hex())

new_mac.hex()

padded: b'actionshowusersolujan@gmail.com\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01x'
len_bytes: 0178
len 64
376


'e02f8dfab0e2cc8ec783399655b4d534553a563f223cf5a39e484ac9a94cb011'

In [66]:
url_respo = 'https://ciberseguridad.diplomatura.unc.edu.ar/cripto/secret-prefix-mac/solujan@gmail.com/answer'
params = dict()
params['a'] = padded[1:]
params['admin'] = 'true'
params['user'] = 'solujan@gmail.com'
params['mac'] = new_mac.hex()


r = requests.get(url_respo, params=params)
print(r.url)
print(r.content)

https://ciberseguridad.diplomatura.unc.edu.ar/cripto/secret-prefix-mac/solujan@gmail.com/answer?a=ctionshowusersolujan%40gmail.com%80%00%00%00%00%00%00%00%00%00%00%00%00%00%00%01x&admin=true&user=solujan%40gmail.com&mac=e02f8dfab0e2cc8ec783399655b4d534553a563f223cf5a39e484ac9a94cb011
b'Lo siento, siga participando.\n'


In [67]:
# MAC = SHA-256(secreto || mensaje)
original_hash = sha256sum.SHA256(b'1234567891234567'+mensaje_bytes).digest()
# padded es el relleno (padding) que debió aplicar la función de hash sobre el mensaje original,
# y en el cual está codificada la longitud del mensaje al cual se aplicó dicha función (lo cual incluye la
# longitud del secreto). Como se trata de SHA-256 la longitud está expresada como un número de 64 bits utilizando
# la convención big endian, ocupando los últimos 8 bytes del relleno.
padded = pad(mensaje_bytes)
# extension es el mensaje agregado.
extension = b'admintrueusersolujan@gmail.com'
# Nuevo_MAC = SHA-256'(extension, MAC, longitud)
new_mac = sha256sum.SHA256(extension, original_hash, len(padded)+16).digest()
# Tiene que ser igual a aplicar sha256 del secreto + padded + extension
expected = sha256sum.SHA256( b'1234567891234567'+padded + extension).digest()
print(original_hash.hex())
print(expected.hex())
print(new_mac.hex())

padded: b'actionshowusersolujan@gmail.com\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01x'
len_bytes: 0178
len 64
376
8f2179558214d52818a6e5f3a30c5c3b7406de9f3b7cd94844ffe40f109221d0
9b7f0c72598c238f37b51842fa7a2ea253dcd0b70ae540fbb7f7b5cd0bb74b7a
9b7f0c72598c238f37b51842fa7a2ea253dcd0b70ae540fbb7f7b5cd0bb74b7a


16